# 1. Загрузка материалов

[Видео, с которым я работаю](https://www.youtube.com/watch?v=J5Pl7bk5G1I)<br>
Для скачивания аудио я использую библиотеку [pytube](https://pytube.io/en/latest/)<br>
Для скачивания субтитров я использую библиотеку [download-youtube-subtitle](https://pypi.org/project/download-youtube-subtitle/)

In [10]:
%%capture
! pip install download-youtube-subtitle
! pip install pytube

In [11]:
from pytube import YouTube
import wave
import librosa
import soundfile as sf
import json

In [12]:
url = 'https://www.youtube.com/watch?v=J5Pl7bk5G1I'

In [13]:
!dl-youtube-cc {url} --to_json -o subtitles.json

INFO:  available caption(s):
INFO:  #0  ✔ as original code="a.es" name="Spanish (auto-generated)" automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  subtitles.json


In [14]:
with open('subtitles.json', 'r', encoding='utf-8') as f:
  subtitles_json = json.load(f)

subtitles = ''
for seg in subtitles_json['original']: # из субтитров оставляю только текст
  if seg['text'] != '\n':
    subtitles += ' ' + seg['text']
subtitles = subtitles[1:]

In [15]:
vid = YouTube(url)
vid.streams.get_audio_only().download(filename='audio.wav')

'/content/audio.wav'

Для дальнейшей работы некоторых моделей понадобится специально перезаписанный WAV файл:

In [ ]:
%%capture
x,_ = librosa.load('audio.wav', sr=16000)
sf.write('tmp.wav', x, 16000)

# 2. Системы ASR

Такие системы, как Whisper, довольно часто зацикливаются и повторяют одно и то же по нескольку раз. Для повторяющихся слов, биграмм и триграмм подготовил функцию очистки:

In [ ]:
def clean(text_inp):
  tokens = text_inp.lower().split()
  i = 1
  while i < len(tokens):
    len_st = len(tokens)
    if tokens[i-1] == tokens[i]:
      tokens.pop(i)
    if i >= 3:
      if tokens[i-3:i-1] == tokens[i-1:i+1]:
        tokens.pop(i)
        tokens.pop(i-1)
    if i >= 5:
      if tokens[i-5:i-2] == tokens[i-2:i+1]:
        tokens.pop(i)
        tokens.pop(i-1)
        tokens.pop(i-2)
    if len(tokens) == len_st:
      i+=1
  return ' '.join(tokens)

## 2.1. HuggingFace **Whisper**

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("automatic-speech-recognition", model="rjac/whisper-tiny-spanish")

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
out = pipe('audio.wav', chunk_length_s=25, stride_length_s=(5, 5))
text_whisper = clean(out['text'])

## 2.2. PocketSphinx **CMU Sphinx ES**

In [ ]:
%%capture
! pip install pocketsphinx

In [ ]:
import pocketsphinx

Испанский модуль для модели sphinx я скачал [отсюда](https://sourceforge.net/projects/cmusphinx/files/Acoustic%20and%20Language%20Models/Spanish/), нужные компоненты загрузил себе на гитхаб в папку с домашкой, далее подгружаю оттуда:

In [ ]:
%%capture
! wget "https://github.com/PhilBurub/NLPcourse_HSE/raw/main/Fall%20'23%20HW2/es-20k.lm.gz"
! wget "https://raw.githubusercontent.com/PhilBurub/NLPcourse_HSE/main/Fall%20'23%20HW2/es.dict"
! wget "https://github.com/PhilBurub/NLPcourse_HSE/raw/main/Fall%20'23%20HW2/voxforge_es_sphinx.cd_ptm_4000.zip"
! unzip voxforge_es_sphinx.cd_ptm_4000.zip

In [ ]:
with wave.open('tmp.wav', "rb") as audio:
    decoder = pocketsphinx.Decoder(lm='es-20k.lm.gz',
                               dict='es.dict',
                               hmm='voxforge_es_sphinx.cd_ptm_4000')
    decoder.start_utt()
    decoder.process_raw(audio.getfp().read(), full_utt=True)
    decoder.end_utt()
    text_sphinx = decoder.hyp().hypstr

## 2.3. SpeechRecognition **SpeechAI ES**

In [ ]:
%%capture
! pip install SpeechRecognition
! pip install google-cloud-speech

In [ ]:
from speech_recognition import Recognizer, AudioData
import speech_recognition as sr
import torchaudio
import json

По инструкции я создал API Key, который я скрыл при публикации тетрадки

In [ ]:
recognizer = Recognizer()
with sr.AudioFile('tmp.wav') as source:
  audio_file = recognizer.record(source)
  text_speechai = recognizer.recognize_google(audio_file,
                                    language='es-ES',
                                    key='MY_KEY_HERE')

# 3. Запись результатов и оценка

In [ ]:
%%capture
! pip install jiwer

In [4]:
import json
from google.colab.files import download, upload
from jiwer import wer
import re

## 3.1. Записываю результаты

In [ ]:
with open('all_texts.json', 'w', encoding='utf-8') as f:
  json.dump({'orig_subtitles': subtitles,
             'whisper': text_whisper,
             'sphinx': text_sphinx,
             'speechai': text_speechai},
            f, ensure_ascii=False, indent='\t')
download('all_texts.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3.2. Загружаю данные и оцениваю

In [ ]:
%%capture
! wget "https://raw.githubusercontent.com/PhilBurub/NLPcourse_HSE/main/Fall%20'23%20HW2/all_texts.json"
! wget "https://raw.githubusercontent.com/PhilBurub/NLPcourse_HSE/main/Fall%20'23%20HW2/etalon.txt"

In [7]:
with open('etalon.txt', 'r', encoding='utf-8') as f:
  reference = f.read()

with open('all_texts.json', 'r', encoding='utf-8') as f:
  all_texts = json.load(f)

In [9]:
for key, text in all_texts.items():
  text_clean = ' '.join(re.findall(r'\w+', text.lower()))
  print('WER for', key, ':', wer(reference, text_clean))

WER for orig_subtitles : 0.1266490765171504
WER for whisper : 0.18469656992084432
WER for sphinx : 0.6464379947229552
WER for speechai : 0.06860158311345646


SpeechAI от Google оказался лидером, далее идут автоматические субтитры, которые генерируются родственной моделью, потом идет tiny whisper, и в конце pocketshinx.